In [1]:
import os

In [2]:
%pwd

'f:\\ProjectAI\\FaceEmotionRecognitionSystem\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'f:\\ProjectAI\\FaceEmotionRecognitionSystem'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class PrepareBaseModelConfig:
    root_dir: Path
    base_model_path: Path
    updated_base_model_path: Path
    params_image_size: list
    params_learning_rate: float
    params_model_name: str
    params_optimizer: str
    params_classes: int

In [6]:
from FacialExpressionRecognition.constants import *
from FacialExpressionRecognition.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH,
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
        config = self.config.prepare_base_model
        params = self.params
        create_directories([config.root_dir])
        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir=Path(config.root_dir),
            base_model_path=Path(os.path.join(config.base_model_path)),
            updated_base_model_path=Path(os.path.join(config.updated_base_model_path)),
            params_image_size=params.IMAGE_SIZE,
            params_learning_rate=params.LEARNING_RATE,
            params_model_name=params.MODEL_NAME,
            params_optimizer=params.OPTIMIZER,
            params_classes=params.CLASSES,
        )
        return prepare_base_model_config

In [13]:

import os
import urllib.request as request
from zipfile import ZipFile
import torch
import torchvision.models as models
import torchvision
from FacialExpressionRecognition import logger

In [15]:
class PrepareBaseModel:
    def __init__(self, config: PrepareBaseModelConfig):
        self.config = config

    def get_model(self, pretrained=True):
        model = torchvision.models.resnet34(pretrained=pretrained)
        model.fc = torch.nn.Linear(model.fc.in_features, self.config.params_classes)
        logger.info(f"Model: {model} - Parameters: {sum(p.numel() for p in model.parameters())}")
        return model


    def update_base_model(self):
        model = self.get_model(pretrained=True)
        self.save_model(model, self.config.updated_base_model_path)

    @staticmethod
    def save_model(model, model_path):
        torch.save(model.state_dict(), model_path)

In [16]:
try:
    config = ConfigurationManager()
    prepare_base_model_config = config.get_prepare_base_model_config()
    prepare_base_model = PrepareBaseModel(prepare_base_model_config)
    prepare_base_model.update_base_model()
except Exception as e:
    raise e


[2025-09-27 21:38:33,411: INFO: common]: yaml file: config\config.yaml loaded successfully
[2025-09-27 21:38:33,414: INFO: common]: yaml file: params.yaml loaded successfully
[2025-09-27 21:38:33,415: INFO: common]: Directory created at: artifacts
[2025-09-27 21:38:33,415: INFO: common]: Directory created at: artifacts/prepare_base_model
[2025-09-27 21:38:33,767: INFO: 1567097251]: Model: ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=